Its a database of the salary and benefits paid to City employees since fiscal year 2013. This data is summarized and presented on the Employee Compensation report hosted at http://openbook.sfgov.org. The features of dataset are as given below - 

Year Type - Fiscal (July through June) or Calendar (January through December) 

Year - An accounting period of 12 months. The City and County of San Francisco operates on a fiscal year that begins on July 1 and ends on June 30 the following year. The Fiscal Year ending June 30, 2012 is represented as FY 2011-2012. 

Organization Group Code - Org Group is a group of Departments. For example, the Public Protection Org Group includes departments such as the Police, Fire, Adult Probation, District Attorney, and Sheriff. 

Organization Group - Org Group is a group of Departments. For example, the Public Protection Org Group includes departments such as the Police, Fire, Adult Probation, District Attorney, and Sheriff. 

Department Code- Departments are the primary organizational unit used by the City and County of San Francisco. Examples include Recreation and Parks, Public Works, and the Police Department. 

Department - Departments are the primary organizational unit used by the City and County of San Francisco. Examples include Recreation and Parks, Public Works, and the Police Department. 

Union Code-Unions represent employees in collective bargaining agreements. A job belongs to one union, although some jobs are unrepresented (usually temporarily). 

Union - Unions represent employees in collective bargaining agreements. A job belongs to one union, although some jobs are unrepresented (usually temporarily). 

Job Family Code - Job Family combines similar Jobs into meaningful groups.

 Job Family - Job Family combines similar Jobs into meaningful groups. 
 
 Job Code - Jobs are defined by the Human Resources classification unit. Examples include gardeners, police officers, and accountants. Job - Jobs are defined by the Human Resources classification unit. Examples include gardeners, police officers, and accountants.
 
  Employee Identifier - Each distinct number in the “Employee Identifier” column represents one employee. These identifying numbers are not meaningful but rather are randomly assigned for the purpose of building this dataset. The column does not appear on the Employee Compensation report hosted on openbook.sfgov.org, but that report does show one row for each employee. Employee ID has been included here to allow users to reconstruct the original report. Note that each employee’s identifier will change each time this dataset is updated, so comparisons by employee across multiple versions of the dataset are not possible. 
  
  Overtime - Amounts paid to City employees working in excess of 40 hours per week. 
  
  Other Salaries - Various irregular payments made to City employees including premium pay, incentive pay, or other one-time payments. 
  
  Retirement - City contributions to employee retirement plans. 
  
  Health/Dental - City-paid premiums to health and dental insurance plans covering City employees. To protect confidentiality as legally required, pro-rated citywide averages are presented in lieu of employee-specific health and dental benefits. 
  
  Other Benefits - Mandatory benefits paid on behalf of employees, such as Social Security (FICA and Medicare) contributions, unemployment insurance premiums, and minor discretionary benefits not included in the above categories. 
  
  Total Benefits - The sum of all benefits paid to City employees. Class- Class of Normal salaries paid to permanent or temporary City employees. Note: In the Class variable - 1 : Low range salary 2 : Mid range salary 3 : High range salary 

In [56]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [57]:
df = pd.read_csv("train.csv")
dff = pd.read_csv("test.csv")
# df.sample(5)

In [58]:
df.drop(columns=['Year Type', 'Organization Group', 'Department', 'Union', 'Job Family', 'Job','ID','Year', 'Union Code', 'Employee Identifier', 'Department Code','Job Family Code','Job Code'], inplace=True)
dff.drop(columns=['Year Type', 'Organization Group', 'Department', 'Union', 'Job Family', 'Job','ID','Year', 'Union Code', 'Employee Identifier', 'Department Code','Job Family Code','Job Code'], inplace=True)
# df.sample(10).T

In [59]:
corr = df.corr()
corr.style.background_gradient(cmap='coolwarm')

,Organization Group Code,Overtime,Other Salaries,Retirement,Health/Dental,Other Benefits,Total Benefits,Class
Organization Group Code,1.000000,-0.301712,-0.226052,-0.243119,-0.231393,0.022912,-0.202476,-0.223910
Overtime,-0.301712,1.000000,0.361632,0.314432,0.308647,0.061956,0.285985,0.288897
Other Salaries,-0.226052,0.361632,1.000000,0.349144,0.216293,0.054032,0.276517,0.317438
Retirement,-0.243119,0.314432,0.349144,1.000000,0.787526,0.671137,0.962205,0.872818
Health/Dental,-0.231393,0.308647,0.216293,0.787526,1.000000,0.606259,0.888821,0.798186
Other Benefits,0.022912,0.061956,0.054032,0.671137,0.606259,1.000000,0.796860,0.657243
Total Benefits,-0.202476,0.285985,0.276517,0.962205,0.888821,0.796860,1.000000,0.889430
Class,-0.223910,0.288897,0.317438,0.872818,0.798186,0.657243,0.889430,1.000000


In [60]:
x_train = df.drop(columns=['Class'])
y_train = df['Class']
x_test = dff

In [61]:
from sklearn import preprocessing
def convert(data):
    number = preprocessing.LabelEncoder()
    df['Organization Group Code'] = number.fit_transform(df['Organization Group Code'])
    df['Overtime'] = number.fit_transform(df['Overtime'])
    df['Other Salaries'] = number.fit_transform(df['Other Salaries'])
    df['Retirement'] = number.fit_transform(df['Retirement'])
    df['Health/Dental'] = number.fit_transform(df['Health/Dental'])
    df['Other Benefits'] = number.fit_transform(df['Other Benefits'])
    df['Total Benefits'] = number.fit_transform(df['Total Benefits'])
    data=data.fillna(0)
    return data

x_train=convert(x_train)
x_test=convert(x_test)

In [64]:
# x_train= np.asarray(x_train)

x_train= np.asarray(x_train).astype(np.float32)
from tensorflow.keras.utils import to_categorical 

y_train1 = to_categorical(y_train)
y_train1=np.asarray(y_train1)
type(x_train)

numpy.ndarray

In [69]:
y_train1

array([[0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       ...,
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.]], dtype=float32)

In [68]:
y_train1 = y_train1[:,1:4]

In [93]:
import tensorflow as tf
from tensorflow.keras.layers import  Dense, Dropout
from tensorflow.keras.models import Sequential
tf.keras.backend.clear_session()
model = Sequential()
# Apply Dropout with 0.2 probability
# Add Fully Connected Layer with 128 units and activation function as 'relu'
model.add(Dense(15000, activation='relu'))
model.add(Dense(1240, activation='relu'))
model.add(Dense(150, activation='relu'))
#Add Fully Connected Layer with 10 units and activation function as 'softmax'
model.add(Dropout(rate=0.25))
model.add(Dense(3, activation='softmax'))
# Compile the model
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
# Use earlystopping
# from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
# callback = tensorflow.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=2, min_delta=0.01)
# Fit the model
model.fit(x=x_train, y=y_train1, batch_size=32, epochs=5) #,  callbacks=[callback])
# let`s train for few more epochs to improve the accuracy
# model.evaluate(X_test, y_test)
# CNN_train_acc = model.evaluate(x_train, y_train)[1]
# CNN_test_acc = model.evaluate(x_test, y_test)[1]
# CNN_loss = model1.evaluate(x_test, y_test)[0]
# final_df.loc[len(final_df.index)] = ['CNN' , CNN_train_acc, CNN_test_acc, CNN_loss]
# final_df

Epoch 1/5
4659/4659 [==============================] - 11225s 2s/step - loss: 10.5468 - accuracy: 0.3549
Epoch 2/5
1406/4659 [========>.....................] - ETA: 9:20 - loss: 1.0970 - accuracy: 0.3444

In [85]:
# model.fit(x_train, y_train, batch_size = 32, epochs= 20, initial_epoch=5)
model.evaluate(x_train, y_train1)
Y_PRED = model.predict(x_test)
y_pred = np.argmax(Y_PRED, axis =1)+1


4659/4659 [==============================] - 9s 2ms/step - loss: 1.0951 - accuracy: 0.3438


In [72]:
type(Y_PRED)

numpy.ndarray

In [88]:
y_pred[50]

3

In [83]:
y_pred[1] = Y_PRED[1].argmax()

TypeError: 'numpy.int64' object does not support item assignment

In [84]:
y_pred[1]

IndexError: invalid index to scalar variable.

In [74]:
y_pred = y_pred+1

In [77]:
y_pred

2941

In [89]:
y_pred1 = pd.DataFrame(y_pred)

In [92]:
y_pred1.to_csv('sol2.csv')

In [ ]:
import pyttsx3
text = pyttsx3.init()
s = 'The code is Compiled'
text.say(s)
text.runAndWait()

In [ ]:

test = pd.read_csv("test.csv")

In [ ]:
submit = test['ID']

In [ ]:
Y_PRED = Y_PRED.argmax()

In [ ]:
type(Y_PRED)

In [ ]:
Y_PRED

In [ ]:
import numpy as np
import pandas as pd

my_array = np.array([[11,22,33],[44,55,66]])
print(type(my_array))
df = pd.DataFrame(my_array, columns = ['Column_A','Column_B','Column_C'])

print(df)
print(type(df))

In [ ]:
y_pred =pd.DataFrame(Y_PRED)
y_pred.sample(6)

In [ ]:
sub = df.append([submit, pd.DataFrame(Y_PRED)])
sub.sample(6)

In [ ]:
submit.to_csv('sub1.csv')

In [ ]:
y_pred = pd.DataFrame(Y_PRED)

In [ ]:
y_pred.head()

In [ ]:
ddf = pd.read_csv('test.csv')

In [ ]:
ddf.sample(10)